<a href="https://colab.research.google.com/github/chendingyan/NLP490H/blob/master/NLP_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Coursework

## Initialization

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip drive/My\ Drive/data/OffensEval_task_data/Test\ C\ Release.zip -d drive/My\ Drive/data/OffensEval_task_data/

Archive:  drive/My Drive/data/OffensEval_task_data/Test C Release.zip
  inflating: drive/My Drive/data/OffensEval_task_data/test_set_taskc.tsv  
  inflating: drive/My Drive/data/OffensEval_task_data/readme-testsetc-v1.txt  


In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm 
import codecs
import random
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, average_precision_score, classification_report,f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.decomposition import TruncatedSVD

from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing import sequence, text
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D,Dropout
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers.merge import concatenate
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download('wordnet')
stop = stopwords.words("english")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


#we fix the seeds to get consistent results

SEED = 234
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Load data

In [0]:
PATH = 'drive/My Drive/data/OffensEval_task_data/offenseval-training-v1.tsv'
df = pd.read_csv(PATH,sep='\t')


print('Indexing word vectors.')
glove_path = 'drive/My Drive/data/glove.6B.300d.txt'
embeddings_index = {}
f = open(glove_path, encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



Indexing word vectors.
Found 400000 word vectors.


# Data Cleaning

In [0]:
import re
def preprocess(df):
    print('-------Remove Stop Word--------')
    stopword_set = set(stopwords.words("english"))

    # convert to lower case and split 
    df.tweet = df.tweet.apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in stopword_set]))

    # keep only words
    pat1 = r'@[A-Za-z0-9]+'
    pat2 =r'[^a-zA-Z\s]'
    pat3 =r"\bURL\b"
    combined_pat = r'|'.join((pat1, pat2,pat3))
    regex_pat = re.compile(combined_pat, flags=re.IGNORECASE)
    df.tweet = df.tweet.str.replace(regex_pat, '')

    # join the cleaned words in a list
    df.tweet.str.join("")
    return df
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,She ask native Americans take is,OFF,UNT,NaN
1,90194,Go home youre drunk MAGA Trump,OFF,TIN,IND
2,16820,Amazon investigating Chinese employees selling...,NOT,NaN,NaN
3,62688,Someone shouldveTaken piece shit volcano,OFF,UNT,NaN
4,43605,Obama wanted liberals amp illegals move red ...,NOT,NaN,NaN


In [0]:
testA_path ='drive/My Drive/data/OffensEval_task_data/testset-taska.tsv'
testB_path = 'drive/My Drive/data/OffensEval_task_data/testset-taskb.tsv'
testC_path ='drive/My Drive/data/OffensEval_task_data/testset-taskb.tsv'

df_a = pd.read_csv(testA_path,sep='\t')
df_a = preprocess(df_a)
df_b = pd.read_csv(testB_path,sep='\t')
df_b = preprocess(df_b)
df_c = pd.read_csv(testC_path,sep='\t')
df_c = preprocess(df_c)

x_test_a = df_a.tweet.values
x_test_b = df_b.tweet.values
x_test_c = df_c.tweet.values


-------Remove Stop Word--------
-------Remove Stop Word--------
-------Remove Stop Word--------


In [0]:
df.to_csv('drive/My Drive/data/OffensEval_task_data/clean_dataset.tsv',encoding='utf-8')

In [0]:
# split data for different subtasks
lbl = LabelEncoder()
y = lbl.fit_transform(df.subtask_a.values)
x = df.tweet.values

# Need data cleaning here!
x_train_a, x_val_a, y_train_a,y_val_a =train_test_split(x,y,stratify=y,random_state=SEED,test_size=0.1,shuffle=True)


taskb_idx= y==1
y_b =df.subtask_b.values[taskb_idx]
y_b = np.where(y_b=='TIN',1,0)
x_b = x[taskb_idx]
x_train_b, x_val_b, y_train_b,y_val_b =train_test_split(x_b,y_b,stratify=y_b,random_state=SEED,test_size=0.1,shuffle=True)


taskc_idx= y_b==1
y_c =df.subtask_c.values[taskb_idx][taskc_idx]
print(y_c)
lbl_enc = preprocessing.LabelEncoder()
y_c = lbl_enc.fit_transform(y_c)
x_c = x_b[taskc_idx]
x_train_c, x_val_c, y_train_c,y_val_c =train_test_split(x_c,y_c,stratify=y_c,random_state=SEED,test_size=0.1,shuffle=True)
# one-hot encode
y_train_c_enc = np_utils.to_categorical(y_train_c)
y_val_c_enc = np_utils.to_categorical(y_val_c)


['IND' 'OTH' 'GRP' ... 'GRP' 'IND' 'OTH']


In [0]:
y_c

array([1, 2, 0, ..., 0, 1, 2])

## Feature extract

In [0]:
max_f=10000

def tf_idf(x_train, x_val,x_test=None):

    tfv = TfidfVectorizer(min_df=3, max_df=0.5, max_features=max_f,ngram_range=(1, 3),use_idf=True,smooth_idf=True)
    tfv.fit(list(x_train) + list(x_val))
    x_train_tfv =  tfv.transform(x_train).todense()
    x_val_tfv = tfv.transform(x_val).todense()
    if x_test is not None:
        x_test_tfv = tfv.transform(x_test).todense()
        return x_train_tfv, x_val_tfv, x_test_tfv
    return x_train_tfv, x_val_tfv
        
        
def bag_of_word(x_train,x_val):
    ctv = CountVectorizer(min_df=3,max_df=0.5,ngram_range=(1,2))
    ctv.fit(list(x_train) + list(x_val))
    x_train_ctv =  ctv.transform(x_train) 
    x_val_ctv = ctv.transform(x_val)
    return x_train_ctv, x_val_ctv

def word_vectors(x):
    N =[]
    for s in tqdm(x):
        M = []
        for w in s:
            try:
                M.append(embeddings_index[w])
            except:
                continue
        M = np.array(M)
        v = M.sum(axis=0)
        if type(v) != np.ndarray:
            return np.zeros(300)
        N.append(v / np.sqrt((v ** 2).sum()))
    N = np.array(N)
    return N


def tokenize_text(train, val):
    token = text.Tokenizer(num_words=None)
 
    tokenized_corpus = [text.text_to_word_sequence(i) for i in train]
    sent_lengths = [len(sent) for sent in tokenized_corpus]
    max_len = np.max(np.array(sent_lengths))
    token.fit_on_texts(train)
    x_train_seq = token.texts_to_sequences(train)
    x_val_seq = token.texts_to_sequences(val)

    # zero padding
    x_train_pad = sequence.pad_sequences(x_train_seq, maxlen=max_len)
    x_val_pad = sequence.pad_sequences(x_val_seq, maxlen=max_len)

    word_index = token.word_index

    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in tqdm(word_index.items()):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    return x_train_pad, x_val_pad, max_len, word_index, embedding_matrix

In [0]:
x_train_tfv, x_val_tfv = tf_idf(x_train_a,x_val_a)
model = Sequential()
model.add(Dense(1024,input_shape=(max_f,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[f1])


batch_size = 64
nb_epochs = 20
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(x_train_tfv, y_train_a, batch_size=batch_size,validation_data=(x_val_tfv, y_val_a),epochs=nb_epochs,verbose=1,callbacks=[earlystop])

Train on 11916 samples, validate on 1324 samples
Epoch 1/20
11916/11916 [==============================] - 5s 450us/step - loss: 0.5933 - f1: 0.2255 - val_loss: 0.5556 - val_f1: 0.4328
Epoch 2/20
11916/11916 [==============================] - 4s 337us/step - loss: 0.4054 - f1: 0.7010 - val_loss: 0.5832 - val_f1: 0.5218
Epoch 3/20
11916/11916 [==============================] - 4s 337us/step - loss: 0.2264 - f1: 0.8672 - val_loss: 0.7491 - val_f1: 0.5448
Epoch 4/20
11916/11916 [==============================] - 4s 337us/step - loss: 0.1004 - f1: 0.9500 - val_loss: 1.0486 - val_f1: 0.5433


## Classical Method 

In [0]:
def logistic_regression(x_train, y_train, x_val, y_val, if_tune=False):
    if if_tune == True:
        clf =logistic_tune(x_train, y_train)
    else:

        clf = LogisticRegression(C=1.0,solver='lbfgs')
        clf.fit(x_train, y_train)
    acc=clf.score(x_val, y_val)
    pred = clf.predict(x_val)
    print('Logistic regression f1 score :', f1_score(y_val,pred,average='macro'))
    print('Logistic regression :',acc)
    return clf
        
        
def naive_bayes(x_train, y_train, x_val, y_val, if_tune=False):
    if if_tune == True:
        clf = naive_bayes(x_train, y_train)
    else:
        clf = MultinomialNB()
        clf.fit(x_train, y_train)
    acc = clf.score(x_val, y_val)
    pred = clf.predict(x_val)
    print('Naive Bayes f1 score :', f1_score(y_val,pred,average='macro'))
    print('Naive Bayes :',acc)
    return clf
        

def svm(x_train, y_train, x_val, y_val, if_tune=False):
    if if_tune == True:
        clf = svm_tune(x_train, y_train)
    else:
        svd = decomposition.TruncatedSVD(n_components=120)
        svd.fit(x_train)
        x_train_svd = svd.transform(x_train)
        x_val_svd = svd.transform(x_val)

        scl = StandardScaler()
        scl.fit(x_train_svd)
        x_train_svd_scl = scl.transform(x_train_svd)
        x_val_svd_scl = scl.transform(x_val_svd)
        clf = SVC(C=1.0, probability=True) # since we need probabilities
        clf.fit(x_train_svd_scl, y_train)
    acc = clf.score(x_val, y_val)
    pred = clf.predict(x_val)
    print('SVM f1 score :', f1_score(y_val,pred,average='macro'))
    print('SVM :',acc)
    return clf


def xgboost(x_train, y_train, x_val, y_val, if_tune=False):
    if if_tune == True:
        clf = xgboost_tune(x_train,y_train)
    else:
        clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                            subsample=0.8, nthread=10, learning_rate=0.1)
        clf.fit(x_train.tocsc(), y_train)
    acc = clf.score(x_val.tocsc(),y_val)
    pred = clf.predict(x_val)
    print('Xgboost f1 score :', f1_score(y_val,pred,average='macro'))
    print('Xgboost :',acc)

In [0]:
x_train_tfv, x_val_tfv = tf_idf(x_train_a, x_val_a)
naive_bayes(x_train_tfv, y_train_a, x_val_tfv, y_val_a)

Naive Bayes f1 score : 0.38030560271646857
Naive Bayes : 0.724320241691843


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Hyperparameter Tuning

In [0]:
x_train_tfv, x_val_tfv = tf_idf(x_train_a, x_val_a)

# Use pipeline for parameter tuning 
# mll_scorer = metrics.make_scorer(accuracy_score, greater_is_better=False)

def svm_tune(x_train_tfv, y_train_a):
    svd = TruncatedSVD()

    # Standard Scaler
    scl = preprocessing.StandardScaler()

    svc = SVC()
    # SVM
    clf = pipeline.Pipeline([('svd',svd), ('scl',scl), ('svm',svc)])

    param_grid = {'svd__n_components':[120,180],
                 'svm__C':[1, 10],
                  'svm__kernel':('linear', 'rbf'),
                  'svm__gamma':['auto','scale']
                 }
    model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy',
                                         verbose=10, n_jobs=-1, iid=True, refit=True)
    model.fit(x_train_tfv, y_train_a) 

    print("Best score: %0.3f" % model.best_score_)

    return model.best_estimator_



#Logistic Regression
def logistic_tune(x_train_tfv, y_train_a):
    lr_model = LogisticRegression()

    # pipeline 
    clf = pipeline.Pipeline([('lr', lr_model)])
    param_grid = {
                         'lr__C': [0.1, 1.0, 10], 
                        'lr__penalty': ['l1', 'l2']}

    model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy',
                                     verbose=10, n_jobs=-1, iid=True, refit=True)
    model.fit(x_train_tfv, y_train_a) 

    print("Best score: %0.3f" % model.best_score_)
    
    return model.best_estimator_

# Naive Bayes
def naive_bayes_tune(x_train_tfv, y_train_a):
    nb_model = MultinomialNB()

    clf = pipeline.Pipeline([('nb', nb_model)])

    param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

    # （Grid Search Model）
    model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy',
                                     verbose=10, n_jobs=-1, iid=True, refit=True,return_train_score=True)

    model.fit(x_train_tfv, y_train_a) 
    print("Best score: %0.3f" % model.best_score_)
    
    return model.best_estimator_

# Xgboost
def xgboost_tune(x_train_tfv, y_train_a):
    
    xgb_model = xgb.XGBClassifier()
    clf = pipeline.Pipeline([('xgb', xgb_model)])
    param_grid = {'learning_rate' :[0.1,0.01,0.001],
                        'n_estimators':[200,400,600,800,1000],
                        'max_depth':[3,5,7],
                        'min_child_weight': [1,3,5]}
    
    model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy',
                                     verbose=10, n_jobs=-1, iid=True, refit=True,return_train_score=True)

    model.fit(x_train_tfv.tocsc(), y_train_a) 
    print("Best score: %0.3f" % model.best_score_)
    
    return model.best_estimator_

In [0]:
nb_model = MultinomialNB()

clf = pipeline.Pipeline([('nb', nb_model)])

param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# （Grid Search Model）
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy',
                                 verbose=10, n_jobs=-1, iid=True, refit=True,return_train_score=True)

model.fit(x_train_tfv, y_train_a) # 为了减少计算量，这里我们仅使用xtrain
sc= model.score( x_val_tfv, y_val_a)
print('valiation score: %0.3f' % sc )
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


valiation score: 0.724
Best score: 0.720
Best parameters set:
	nb__alpha: 1


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1873s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:    1.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    1.4s finished


In [0]:
from hypopt import GridSearch

param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

opt = GridSearch(model= MultinomialNB(), param_grid=param_grid)
opt.fit(x_train_tfv, y_train_a, x_val_tfv, y_val_a, scoring = 'accuracy')
print('Test Score for Optimized Parameters:', opt.score(x_val_tfv, y_val_a))

Test Score for Optimized Parameters: 0.724320241691843


## Embedding

## Deep Neural Network


In [0]:
# from keras import backend as K

# def f1(y_true, y_pred):
#     def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))
import tensorflow as tf
import keras.backend as K

# macro f1 score
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [0]:
#Fully Connected Network
def fcn_model(num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam', metrics=[f1] ):
    fcn_model = Sequential()
    fcn_model.add(Dense(300, input_dim=300, activation='relu'))
    fcn_model.add(Dropout(0.2))
    fcn_model.add(BatchNormalization())

    fcn_model.add(Dense(300, activation='relu'))
    fcn_model.add(Dropout(0.3))
    fcn_model.add(BatchNormalization())
    fcn_model.add(Dense(num_classes, activation=final_activation))

    fcn_model.compile(loss=loss, optimizer=optim, metrics=metrics)
    return fcn_model

In [0]:
# LSTM
def lstm_model(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam', metrics=[f1]):
    lstm_model = Sequential()
    lstm_model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],  input_length=max_len, trainable=False))
    lstm_model.add(SpatialDropout1D(0.3))
    lstm_model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))
    lstm_model.add(Dense(512, activation='relu'))
    lstm_model.add(Dropout(0.3))
    lstm_model.add(Dense(256, activation='relu'))
    lstm_model.add(Dropout(0.3))

    lstm_model.add(Dense(num_classes,activation=final_activation))
    
    lstm_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return lstm_model

In [0]:
# GRU Network
def gru_model(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam',  metrics=[f1]):

    gru_model = Sequential()
    gru_model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],  input_length=max_len, trainable=False))
    gru_model.add(SpatialDropout1D(0.3))
    gru_model.add(GRU(100, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
    gru_model.add(GRU(120, dropout=0.3, recurrent_dropout=0.3))

    gru_model.add(Dense(512, activation='relu'))
    gru_model.add(Dropout(0.3))

    gru_model.add(Dense(256, activation='relu'))
    gru_model.add(Dropout(0.3))
    gru_model.add(Dense(num_classes, activation=final_activation))
                  
    gru_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return gru_model

In [0]:
# CNN network
def cnn_model(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam',  metrics=[f1]):


    cnn_model = Sequential()
    cnn_model.add(Embedding(len(word_index) + 1,300,weights=[embedding_matrix],input_length=max_len, trainable=False))
    cnn_model.add(Convolution1D(256, 3, padding='same'))
    cnn_model.add(MaxPool1D(3,3,padding='same'))
    cnn_model.add(Convolution1D(128, 3, padding='same'))
    cnn_model.add(MaxPool1D(3,3,padding='same'))
    cnn_model.add(Convolution1D(64, 3, padding='same'))
    cnn_model.add(Flatten())
    cnn_model.add(Dropout(0.3))
    cnn_model.add(BatchNormalization()) 
    cnn_model.add(Dense(256,activation='relu'))
    cnn_model.add(Dropout(0.1))
    cnn_model.add(Dense(num_classes),activation =final_activation)
                  
    cnn_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return cnn_model

In [0]:
# TextCNN
def textcnn_model(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam', metrics=['accuracy']):

    main_input = Input(shape=(max_len,), dtype='float64')
    embedder = Embedding(len(word_index) + 1, 300, input_length = max_len, weights = [embedding_matrix], trainable = False)
    embed = embedder(main_input)
    cnn1 = Convolution1D(256, 3, padding='same', strides = 1, activation='relu')(embed)
    cnn1 = MaxPool1D(pool_size=4)(cnn1)
    cnn2 = Convolution1D(256, 4, padding='same', strides = 1, activation='relu')(embed)
    cnn2 = MaxPool1D(pool_size=4)(cnn2)
    cnn3 = Convolution1D(256, 5, padding='same', strides = 1, activation='relu')(embed)
    cnn3 = MaxPool1D(pool_size=4)(cnn3)

    # Combine three model into one
    cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(num_classes, activation=final_activation)(drop)
    textcnn_model = Model(inputs = main_input, outputs = main_output)                
    textcnn_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return textcnn_model


In [0]:
# CNN+RNN
def crnn_model(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam',  metrics=[f1]):

    main_input = Input(shape=(max_len,), dtype='float64')
    embed = Embedding(len(word_index)+1, 300, input_length=max_len)(main_input)
    cnn = Convolution1D(256, 3, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPool1D(pool_size=4)(cnn)
    cnn = Flatten()(cnn)
    cnn = Dense(256)(cnn)
    cnn = Dropout(0.3)(cnn)
    rnn = Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1))(embed)
    rnn = Dense(256)(rnn)
    rnn = Dropout(0.3)(rnn)
    con = concatenate([cnn,rnn], axis=-1)
    main_output = Dense(num_classes, activation=final_activation)(con)
    crnn_model = Model(inputs = main_input, outputs = main_output)                
    crnn_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return crnn.model

In [0]:
# Extended textCNN
def etextcnn_model(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam',  metrics=['accuracy']):


    main_input = Input(shape=(max_len,), dtype='float64')
    embedder = Embedding(len(word_index) + 1, 300, input_length = max_len, weights = [embedding_matrix], trainable = False)
    embed = embedder(main_input)

    # cnn1，kernel_size = 3
    conv1_1 = Conv1D(256, 3, padding='same')(embed)
    bn1_1 = BatchNormalization()(conv1_1)
    relu1_1 = Activation('relu')(bn1_1)
    drop1_1 = Dropout(0.3)(relu1_1)
    conv1_2 = Conv1D(128, 3, padding='same')(drop1_1)
    bn1_2 = BatchNormalization()(conv1_2)
    relu1_2 = Activation('relu')(bn1_2)
    drop1_2 = Dropout(0.3)(relu1_2)
    cnn1 = MaxPooling1D(pool_size=4)(drop1_1)
    # cnn2，kernel_size = 4
    conv2_1 = Conv1D(256, 4, padding='same')(embed)
    bn2_1 = BatchNormalization()(conv2_1)
    relu2_1 = Activation('relu')(bn2_1)
    drop2_1 = Dropout(0.3)(relu2_1)
    conv2_2 = Conv1D(128, 4, padding='same')(drop2_1)
    bn2_2 = BatchNormalization()(conv2_2)
    relu2_2 = Activation('relu')(bn2_2)
    drop2_2 = Dropout(0.3)(relu2_2)
    cnn2 = MaxPooling1D(pool_size=4)(drop2_2)
    # cnn3，kernel_size = 5
    conv3_1 = Conv1D(256, 5, padding='same')(embed)
    bn3_1 = BatchNormalization()(conv3_1)
    relu3_1 = Activation('relu')(bn3_1)
    drop3_1 = Dropout(0.3)(relu3_1)
    conv3_2 = Conv1D(128, 5, padding='same')(drop3_1)
    bn3_2 = BatchNormalization()(conv3_2)
    relu3_2 = Activation('relu')(bn3_2)
    drop3_2 = Dropout(0.3)(relu3_2)
    cnn3 = MaxPooling1D(pool_size=4)(drop3_2)

    # Combine three block
    cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.5)(flat)
    fc = Dense(512)(drop)
    bn = BatchNormalization()(fc)
 
    main_output = Dense(num_classes, activation=final_activation)(bn)
    etextcnn_model = Model(inputs = main_input, outputs = main_output)                
    etextcnn_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return etextcnn_model
    

In [0]:
def simple_visualize(model):
    plt.subplot(211)
    plt.title("Accuracy")
    plt.plot(history.history["acc"], color="g", label="Train")
    plt.plot(history.history["val_acc color="b", label="Test")
    plt.legend(loc="best")

    plt.subplot(212)
    plt.title("Loss")
    plt.plot(result.history["loss"], color="g", label="Train")
    plt.plot(result.history["val_loss"], color="b", label="Test")
    plt.legend(loc="best")

    plt.tight_layout()
    plt.show()

## Model Tuning 

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier

x_train_pad, x_val_pad, max_len, word_index, embedding_matrix = tokenize_text(x_train_a,x_val_a)

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta, Adam, rmsprop
    
space = {
            'units1': hp.choice('units1', [16,32,64,128,256,512]),
            'units2': hp.choice('units2', [16,32,64,128,256,512]),
            'units3': hp.choice('units3', [16,32,64,128,256,512]),
            'units4': hp.choice('units4', [16,32,64,128,256,512]),
            'dropout1': hp.uniform('dropout1', .25,.75),
            'dropout2': hp.uniform('dropout2',  .25,.75),
            'dropout3': hp.uniform('dropout3',  .25,.75),
            'dropout4': hp.uniform('dropout4',  .25,.75),

            'batch_size' : hp.choice('batch_size', [32,64,128,256,512]),

            'epochs' :  3,
            'optimizer': hp.choice('optimizer',['sgd','adadelta','adam','rmsprop']),
        }

def gru_tune(params):   

    # GRU Network

    gru_model = Sequential()
    gru_model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],  input_length=max_len, trainable=False))
    gru_model.add(SpatialDropout1D(0.3))
    gru_model.add(GRU(params['units1'], dropout=params['dropout1'], recurrent_dropout=0.3, return_sequences=True))
    gru_model.add(GRU(params['units2'], dropout=params['dropout2'], recurrent_dropout=0.3))

    gru_model.add(Dense(params['units3'], activation='relu',kernel_initializer = "glorot_uniform"))
    gru_model.add(Dropout(params['dropout3']))

    gru_model.add(Dense(params['units4'], activation='relu',kernel_initializer = "glorot_uniform"))
    gru_model.add(Dropout(params['dropout4']))
    gru_model.add(Dense(1, activation='sigmoid'))
                  
    gru_model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])
    gru_model.fit(x_train_pad, y_train_a, epochs=params['epochs'], batch_size=params['batch_size'], verbose = 0)    
    loss = gru_model.evaluate(x_val_pad, y_val_a)
    print(loss)
    return {'loss': loss[0], 'status': STATUS_OK}

def lstm_tune(params):
    lstm_model = Sequential()
    lstm_model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],  input_length=max_len, trainable=False))
    lstm_model.add(SpatialDropout1D(0.3))
    lstm_model.add(Bidirectional(LSTM(params['units1'], dropout=params['dropout1'], recurrent_dropout=0.3)))
    lstm_model.add(Dense(params['units2'], activation='relu'))
    lstm_model.add(Dropout(params['dropout2']))
    lstm_model.add(Dense(params['units3'], activation='relu'))
    lstm_model.add(Dropout(params['dropout3']))

    lstm_model.add(Dense(1, activation='sigmoid'))
    
    lstm_model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])
    lstm_model.fit(x_train_pad, y_train_a, epochs=params['epochs'], batch_size=params['batch_size'], verbose = 0)    
    loss = lstm_model.evaluate(x_val_pad, y_val_a)
    print('validation accuracy :', loss[1])
    return {'loss': -loss[1], 'status': STATUS_OK}

def etextcnn_tune(params):
    main_input = Input(shape=(max_len,), dtype='float64')
    embedder = Embedding(len(word_index) + 1, 300, input_length = max_len, weights = [embedding_matrix], trainable = False)
    embed = embedder(main_input)

    # cnn1，kernel_size = 3
    conv1_1 = Conv1D(params['units1'], 3, padding='same')(embed)
    bn1_1 = BatchNormalization()(conv1_1)
    relu1_1 = Activation('relu')(bn1_1)
    drop1_1 = Dropout(params['dropout1'])(relu1_1)
    conv1_2 = Conv1D(params['units2'], 3, padding='same')(drop1_1)
    bn1_2 = BatchNormalization()(conv1_2)
    relu1_2 = Activation('relu')(bn1_2)
    drop1_2 = Dropout(params['dropout2'])(relu1_2)
    cnn1 = MaxPooling1D(pool_size=4)(drop1_1)
    # cnn2，kernel_size = 4
    conv2_1 = Conv1D(params['units1'], 4, padding='same')(embed)
    bn2_1 = BatchNormalization()(conv2_1)
    relu2_1 = Activation('relu')(bn2_1)
    drop2_1 = Dropout(params['dropout1'])(relu2_1)
    conv2_2 = Conv1D(params['units2'], 4, padding='same')(drop2_1)
    bn2_2 = BatchNormalization()(conv2_2)
    relu2_2 = Activation('relu')(bn2_2)
    drop2_2 = Dropout(params['dropout2'])(relu2_2)
    cnn2 = MaxPooling1D(pool_size=4)(drop2_2)
    # cnn3，kernel_size = 5
    conv3_1 = Conv1D(params['units1'], 5, padding='same')(embed)
    bn3_1 = BatchNormalization()(conv3_1)
    relu3_1 = Activation('relu')(bn3_1)
    drop3_1 = Dropout(params['dropout1'])(relu3_1)
    conv3_2 = Conv1D(params['units2'], 5, padding='same')(drop3_1)
    bn3_2 = BatchNormalization()(conv3_2)
    relu3_2 = Activation('relu')(bn3_2)
    drop3_2 = Dropout(params['dropout2'])(relu3_2)
    cnn3 = MaxPooling1D(pool_size=4)(drop3_2)

    # Combine three block
    cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(params['dropout3'])(flat)
    fc = Dense(params['units3'])(drop)
    bn = BatchNormalization()(fc)
 
    main_output = Dense(1, activation='sigmoid')(bn)
    etextcnn_model = Model(inputs = main_input, outputs = main_output)                
    
    etextcnn_model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])
    etextcnn_model.fit(x_train_pad, y_train_a, epochs=params['epochs'], batch_size=params['batch_size'], verbose = 0)    
    loss = etextcnn_model.evaluate(x_val_pad, y_val_a)
    print('validation accuracy :', loss[1])
    return {'loss': loss[0], 'status': STATUS_OK}

def rcnn_tune(params):
    main_input = Input(shape=(max_len,), dtype='float64')
    embed = Embedding(len(word_index)+1, 300, input_length=max_len)(main_input)
    cnn = Conv1D(params['units1'], 3, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPooling1D(pool_size=4)(cnn)
    cnn = Flatten()(cnn)
    cnn = Dense(params['units2'])(cnn)
    cnn = Dropout(params['dropout1'])(cnn)
    rnn = Bidirectional(GRU(params['units1'], dropout=0.2, recurrent_dropout=0.1))(embed)
    rnn = Dense(params['units3'])(rnn)
    rnn = Dropout(params['dropout2'])(rnn)
    con = concatenate([cnn,rnn], axis=-1)
    main_output = Dense(1, activation='sigmoid')(con)
    crnn_model = Model(inputs = main_input, outputs = main_output)                
    crnn_model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])

    crnn_model.fit(x_train_pad, y_train_a, epochs=params['epochs'], batch_size=params['batch_size'], verbose = 0)    
    loss = crnn_model.evaluate(x_val_pad, y_val_a)
    print('validation accuracy :', loss[1])
    return {'loss': loss[0], 'status': STATUS_OK}


def cnn_tune(params):
    cnn_model = Sequential()
    cnn_model.add(Embedding(len(word_index) + 1,300,weights=[embedding_matrix],input_length=max_len, trainable=False))
    cnn_model.add(Conv1D(params['units1'], 3, padding='same'))
    cnn_model.add(MaxPooling1D(3,3,padding='same'))
    cnn_model.add(Conv1D(params['units2'], 3, padding='same'))
    cnn_model.add(MaxPooling1D(3,3,padding='same'))
    cnn_model.add(Conv1D(params['units3'], 3, padding='same'))
    cnn_model.add(Flatten())
    cnn_model.add(Dropout(params['dropout1']))
    cnn_model.add(BatchNormalization()) 
    cnn_model.add(Dense(params['units4'],activation='relu'))
    cnn_model.add(Dropout(params['dropout2']))
    cnn_model.add(Dense(1,activation ='sigmoid'))
    
    cnn_model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])

    cnn_model.fit(x_train_pad, y_train_a, epochs=params['epochs'], batch_size=params['batch_size'], verbose = 0)    
    loss = cnn_model.evaluate(x_val_pad, y_val_a)
    print('validation accuracy :', loss)
    return {'loss': -loss[1], 'status': STATUS_OK}

trials = Trials()
best = fmin(lstm_tune, space, algo=tpe.suggest, max_evals=50, trials=trials)
print('best:', best)


In [0]:
def cnn_model_opti(word_index, max_len, embedding_matrix, num_classes =1, final_activation = 'sigmoid', loss = 'binary_crossentropy', optim='adam', metrics=['accuracy']):


    cnn_model = Sequential()
    cnn_model.add(Embedding(len(word_index) + 1,300,weights=[embedding_matrix],input_length=max_len, trainable=False))
    cnn_model.add(Conv1D(512, 3, padding='same'))
    cnn_model.add(MaxPooling1D(3,3,padding='same'))
    cnn_model.add(Conv1D(16, 3, padding='same'))
    cnn_model.add(MaxPooling1D(3,3,padding='same'))
    cnn_model.add(Conv1D(32, 3, padding='same'))
    cnn_model.add(Flatten())
    cnn_model.add(Dropout(0.74))
    cnn_model.add(BatchNormalization()) 
    cnn_model.add(Dense(32,activation='relu'))
    cnn_model.add(Dropout(0.38))
    cnn_model.add(Dense(num_classes,activation =final_activation))
                  
    cnn_model.compile(loss=loss, optimizer=optim,metrics=metrics)
    return cnn_model

In [0]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

model = cnn_model_opti(word_index,max_len,embedding_matrix)
model.fit(x_train_pad, y=y_train_a, batch_size=256, epochs=20, 
          verbose=1, validation_data=(x_val_pad, y_val_a), callbacks=[earlystop])

Train on 11916 samples, validate on 1324 samples
Epoch 1/20
11916/11916 [==============================] - 25s 2ms/step - loss: 0.7834 - acc: 0.5719 - val_loss: 0.5851 - val_acc: 0.6994
Epoch 2/20
11916/11916 [==============================] - 2s 154us/step - loss: 0.6244 - acc: 0.6944 - val_loss: 0.5417 - val_acc: 0.7266
Epoch 3/20
11916/11916 [==============================] - 2s 148us/step - loss: 0.5444 - acc: 0.7444 - val_loss: 0.5383 - val_acc: 0.7538
Epoch 4/20
11916/11916 [==============================] - 2s 148us/step - loss: 0.4922 - acc: 0.7784 - val_loss: 0.5289 - val_acc: 0.7508
Epoch 5/20
11916/11916 [==============================] - 2s 150us/step - loss: 0.4365 - acc: 0.8054 - val_loss: 0.5765 - val_acc: 0.7508
Epoch 6/20
11916/11916 [==============================] - 2s 150us/step - loss: 0.3547 - acc: 0.8512 - val_loss: 0.5758 - val_acc: 0.7409
Epoch 7/20
11916/11916 [==============================] - 2s 151us/step - loss: 0.2767 - acc: 0.8887 - val_loss: 0.6066 - va

# Sub_task A

In [0]:
x_train_pad, x_val_pad, max_len, word_index, embedding_matrix = tokenize_text(x_train_a,x_val_a)
_,x_test_pad,_,_,_= tokenize_text(x_train_a,x_test_a)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

model = gru_model(word_index,max_len,embedding_matrix)
model.fit(x_train_pad, y=y_train_a, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_val_pad, y_val_a), callbacks=[earlystop])
pred= model.predict(x_test_pad)
pred=np.where(pred>=0.5,1,0).reshape(-1,)
f1_score(np.zeros(len(x_test_pad)),pred, average='macro')

100%|██████████| 19037/19037 [00:00<00:00, 433359.74it/s]


Train on 11916 samples, validate on 1324 samples
Epoch 1/100
11916/11916 [==============================] - 13s 1ms/step - loss: 0.6251 - f1: 0.0827 - val_loss: 0.5721 - val_f1: 0.4704
Epoch 2/100
11916/11916 [==============================] - 6s 469us/step - loss: 0.5620 - f1: 0.4735 - val_loss: 0.5589 - val_f1: 0.5323
Epoch 3/100
11916/11916 [==============================] - 6s 470us/step - loss: 0.5348 - f1: 0.5066 - val_loss: 0.5239 - val_f1: 0.5592
Epoch 4/100
11916/11916 [==============================] - 6s 466us/step - loss: 0.5164 - f1: 0.5682 - val_loss: 0.5139 - val_f1: 0.5810
Epoch 5/100
11916/11916 [==============================] - 6s 470us/step - loss: 0.5068 - f1: 0.5754 - val_loss: 0.5102 - val_f1: 0.5762
Epoch 6/100
11916/11916 [==============================] - 6s 472us/step - loss: 0.4916 - f1: 0.5935 - val_loss: 0.5091 - val_f1: 0.5851
Epoch 7/100
11916/11916 [==============================] - 6s 471us/step - loss: 0.4898 - f1: 0.5893 - val_loss: 0.5065 - val_f1: 

In [0]:
# model.evaluate(x_val_pad, y_val_a)
pred= model.predict(x_test_pad)
pred=np.where(pred>=0.5,1,0).reshape(-1,)


In [0]:
f1_score(np.zeros(len(x_test_pad)),pred, average='macro')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.4511805998723676

In [0]:
x_train_tfv, x_val_tfv, x_test_tfv = tf_idf(x_train_a, x_val_a,x_test_a)
clf =logistic_regression(x_train_tfv,y_train_a, x_val_tfv, y_val_a)
pred= clf.predict(x_test_tfv)
f1_score(np.zeros(len(x_test_a)),pred, average='macro')

Logistic regression f1 score : 0.6505225610843589
Logistic regression : 0.7484894259818731


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.46716232961586124

In [0]:
x_train_w2v,x_val_w2v = word_count(x_train_a,x_val_a)
svm(x_train_w2v,y_train_a, x_val_w2v, y_val_a)
naive_bayes(x_train_w2v,y_train_a, x_val_w2v, y_val_a)
logistic_regression(x_train_w2v,y_train_a, x_val_w2v, y_val_a)


SVM : 0.7061933534743202
Naive Bayes : 0.716012084592145
Logistic regression : 0.7409365558912386


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
model = etextcnn_model(word_index,max_len,embedding_matrix)
model.fit(x_train_pad, y=y_train_a, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_val_pad, y_val_a), callbacks=[earlystop])
pred= model.predict(x_test_pad)
pred=np.where(pred>=0.5,1,0).reshape(-1,)
f1_score(np.zeros(len(x_test_pad)),pred, average='macro')

Train on 11916 samples, validate on 1324 samples
Epoch 1/100
11916/11916 [==============================] - 14s 1ms/step - loss: 1.3878 - acc: 0.6164 - val_loss: 1.2405 - val_acc: 0.6813
Epoch 2/100
11916/11916 [==============================] - 5s 393us/step - loss: 0.6044 - acc: 0.7349 - val_loss: 0.6317 - val_acc: 0.7258
Epoch 3/100
11916/11916 [==============================] - 5s 397us/step - loss: 0.5061 - acc: 0.7708 - val_loss: 0.6321 - val_acc: 0.7258
Epoch 4/100
11916/11916 [==============================] - 5s 393us/step - loss: 0.4708 - acc: 0.7849 - val_loss: 0.6212 - val_acc: 0.7319
Epoch 5/100
11916/11916 [==============================] - 5s 394us/step - loss: 0.4382 - acc: 0.8038 - val_loss: 0.6879 - val_acc: 0.7175
Epoch 6/100
11916/11916 [==============================] - 5s 393us/step - loss: 0.4076 - acc: 0.8162 - val_loss: 0.7296 - val_acc: 0.7190
Epoch 7/100
11916/11916 [==============================] - 5s 393us/step - loss: 0.3770 - acc: 0.8374 - val_loss: 0.78

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.4837935174069628

# Sub_task B



In [0]:
x_train_tfv, x_val_tfv, x_test_tfv = tf_idf(x_train_b, x_val_b,x_test_b)
clf =logistic_regression(x_train_tfv,y_train_b, x_val_tfv, y_val_b)
pred= clf.predict(x_test_tfv)
f1_score(np.ones(len(x_test_b)),pred, average='macro')

Logistic regression f1 score : 0.487041520939826
Logistic regression : 0.8818181818181818


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.4989561586638831

In [0]:
x_train_pad, x_val_pad, max_len, word_index, embedding_matrix = tokenize_text(x_train_b,x_val_b)
_,x_test_pad,_,_,_= tokenize_text(x_train_b,x_test_b)
model = lstm_model(word_index,max_len,embedding_matrix)
model.fit(x_train_pad, y=y_train_b, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_val_pad, y_val_b), callbacks=[earlystop])
pred= model.predict(x_test_pad)
pred=np.where(pred>=0.5,1,0).reshape(-1,)
f1_score(np.ones(len(x_test_pad)),pred, average='macro')


100%|██████████| 10007/10007 [00:00<00:00, 389904.13it/s]


Train on 3960 samples, validate on 440 samples
Epoch 1/100
3960/3960 [==============================] - 13s 3ms/step - loss: 0.4700 - f1: 0.8965 - val_loss: 0.4065 - val_f1: 0.9372
Epoch 2/100
3960/3960 [==============================] - 2s 546us/step - loss: 0.3827 - f1: 0.9365 - val_loss: 0.3654 - val_f1: 0.9372
Epoch 3/100
3960/3960 [==============================] - 2s 528us/step - loss: 0.3576 - f1: 0.9366 - val_loss: 0.3438 - val_f1: 0.9372
Epoch 4/100
3960/3960 [==============================] - 2s 527us/step - loss: 0.3485 - f1: 0.9366 - val_loss: 0.3461 - val_f1: 0.9372
Epoch 5/100
3960/3960 [==============================] - 2s 528us/step - loss: 0.3402 - f1: 0.9365 - val_loss: 0.3479 - val_f1: 0.9372
Epoch 6/100
3960/3960 [==============================] - 2s 530us/step - loss: 0.3402 - f1: 0.9366 - val_loss: 0.3466 - val_f1: 0.9372


1.0

In [0]:
x_train_pad, x_val_pad, max_len, word_index, embedding_matrix = tokenize_text(x_train_b,x_val_b)
_,x_test_pad,_,_,_= tokenize_text(x_train_b,x_test_b)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model = etextcnn_model(word_index,max_len,embedding_matrix)
model.fit(x_train_pad, y=y_train_b, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_val_pad, y_val_b), callbacks=[earlystop])
pred= model.predict(x_test_pad)
pred=np.where(pred>=0.5,1,0).reshape(-1,)
f1_score(np.ones(len(x_test_pad)),pred, average='macro')

100%|██████████| 10007/10007 [00:00<00:00, 467107.37it/s]


Train on 3960 samples, validate on 440 samples
Epoch 1/100
3960/3960 [==============================] - 13s 3ms/step - loss: 2.3099 - acc: 0.5886 - val_loss: 1.4039 - val_acc: 0.4273
Epoch 2/100
3960/3960 [==============================] - 1s 354us/step - loss: 1.2392 - acc: 0.6381 - val_loss: 0.5397 - val_acc: 0.8795
Epoch 3/100
3960/3960 [==============================] - 1s 355us/step - loss: 0.8273 - acc: 0.7063 - val_loss: 0.5826 - val_acc: 0.7364
Epoch 4/100
3960/3960 [==============================] - 1s 354us/step - loss: 0.7141 - acc: 0.7672 - val_loss: 0.5014 - val_acc: 0.8795
Epoch 5/100
3960/3960 [==============================] - 1s 354us/step - loss: 0.6213 - acc: 0.8306 - val_loss: 0.4214 - val_acc: 0.8795
Epoch 6/100
3960/3960 [==============================] - 1s 354us/step - loss: 0.5643 - acc: 0.8510 - val_loss: 0.3715 - val_acc: 0.8795
Epoch 7/100
3960/3960 [==============================] - 1s 354us/step - loss: 0.4990 - acc: 0.8591 - val_loss: 0.3483 - val_acc: 0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.4989561586638831

# Subtask C

In [0]:
x_train_tfv, x_val_tfv, x_test_tfv = tf_idf(x_train_c, x_val_c,x_test_c)
clf =logistic_regression(x_train_tfv,y_train_c, x_val_tfv, y_val_c)
pred= clf.predict(x_test_tfv)
print(pred)
f1_score(np.ones(len(x_test_c)),pred, average='macro')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic regression f1 score : 0.4442148960250622
Logistic regression : 0.6881443298969072
[0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0
 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0
 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.42857142857142855

In [0]:
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(x_train_tfv, y_train_c_enc)


ValueError: ignored

In [0]:
x_test_c[1]

'nopasaran unity demo oppose farright london  antifa oct  enough enough '